In [ ]:
import geopandas
import matplotlib.pyplot as plt
import shapely

from geodatasets import get_path

Load data on New York City subboroughs to illustrate the spatial
indexing.

In [ ]:
nyc = geopandas.read_file(get_path("geoda nyc"))

## R-tree principle

In principle, any R-tree index builds a hierarchical collection of
bounding boxes (envelopes) representing first individual geometries and then their
most efficient combinations (from a spatial query perspective). When
creating one, you can imagine that your geometries are represented by
their envelopes, as illustrated below.

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(8, 4))

nyc.plot(ax=axs[0], edgecolor="black", linewidth=1)
nyc.envelope.boundary.plot(ax=axs[1], color='black');

In [ ]:
point_inside = shapely.Point(950000, 155000)
point_outside = shapely.Point(1050000, 150000)
points = geopandas.GeoSeries([point_inside, point_outside], crs=nyc.crs)

You can verify that visually.

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(8, 4))

nyc.plot(ax=axs[0], edgecolor="black", linewidth=1)
nyc.envelope.boundary.plot(ax=axs[1], color='black')
points.plot(ax=axs[0], color="limegreen")
points.plot(ax=axs[1], color="limegreen");

In [ ]:
bbox_query_inside = nyc.sindex.query(point_inside)
bbox_query_outside = nyc.sindex.query(point_outside)
bbox_query_inside, bbox_query_outside

Both the point we know is inside a geometry and the one that is outside
a geometry return one hit as each intersects one bounding box in the
tree.

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(8, 4))

nyc.plot(ax=axs[0], edgecolor="black", linewidth=1)
nyc.envelope.boundary.plot(ax=axs[1], color='black')
points.plot(ax=axs[0], color="limegreen", zorder=3, edgecolor="black", linewidth=.5)
points.plot(ax=axs[1], color="limegreen", zorder=3, edgecolor="black", linewidth=.5)
nyc.iloc[bbox_query_inside].plot(ax=axs[0], color='orange')
nyc.iloc[bbox_query_outside].plot(ax=axs[0], color='orange')
nyc.envelope.iloc[bbox_query_inside].plot(ax=axs[1], color='orange')
nyc.envelope.iloc[bbox_query_outside].plot(ax=axs[1], color='orange');

The image above provides a clear illustration of what happens. While you
can see on the left image that only one intersects an orange geometry
marked as a *hit*, the hits are quite clear when looking at the bounding
box.

Thankfully, the spatial index allows for further filtering based on the
actual geometry. In this case, the tree is first queried as above but
afterwards, each of the possible hits is checked using a spatial predicate.

In [ ]:
pred_inside = nyc.sindex.query(point_inside, predicate="intersects")
pred_outside = nyc.sindex.query(point_outside, predicate="intersects")
pred_inside, pred_outside

When you specify ``predicate="intersects"``, the result is indeed
different and the output of the query using the point that lies outside
of any geometry is empty.

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(8, 4))

nyc.plot(ax=axs[0], edgecolor="black", linewidth=1)
nyc.envelope.boundary.plot(ax=axs[1], color='black')
points.plot(ax=axs[0], color="limegreen", zorder=3, edgecolor="black", linewidth=.5)
points.plot(ax=axs[1], color="limegreen", zorder=3, edgecolor="black", linewidth=.5)
nyc.iloc[pred_inside].plot(ax=axs[0], color='orange')
nyc.envelope.iloc[pred_inside].plot(ax=axs[1], color='orange');

In [ ]:
nyc.sindex.valid_query_predicates

In [ ]:
bbox_array_query = nyc.sindex.query(points)
bbox_array_query

By default, the method returns a 2-D array of indices where the query
found a hit where the subarrays correspond to the indices of the input
geometries and indices of the tree geometries associated with each. In
the example above, the 0-th geometry in the ``points`` GeoSeries
intersects the bounding box of the geometry at the position 1 from the
``nyc`` GeoDataFrame, while the geometry 1 in the ``points`` matches
geometry 16 in the ``nyc``. You may notice that these are the same
indices as you’ve seen above.

The other option is to return a boolean array with shape
``(len(tree), n)`` with boolean values marking whether the bounding box
of a geometry in the tree intersects a bounding box of a given geometry.
This can be either a dense numpy array, or a sparse scipy array. Keep in
mind that the output will be, in most cases, mostly filled with
``False`` and the array can become really large, so it is recommended to
use the sparse format, if possible.

You can specify each using the ``output_format`` keyword:

In [ ]:
bbox_array_query_dense = nyc.sindex.query(points, output_format="dense")
bbox_array_query_dense

In [ ]:
bbox_array_query_sparse = nyc.sindex.query(points, output_format="sparse")
bbox_array_query_sparse

For example, to find the number of neighboring geometries for each subborough, you can
use the spatial index to compare all geometries against each other. Since you are using
``nyc`` on both sides of the query here, the resulting array is square-shaped with
diagonal filled with ``True``.

In [ ]:
neighbors = nyc.sindex.query(nyc.geometry, predicate="intersects", output_format="dense")
neighbors

Getting the sum along one axis can then give you the answer. Note that
since a geometry always intersects itself, you need to subtract one.

In [ ]:
n_neighbors = neighbors.sum(axis=1) - 1
n_neighbors

The result is a numpy array you can directly plot on a map.

In [ ]:
nyc.plot(n_neighbors, legend=True);

### Nearest geometry query

While checking the spatial predicate using the spatial index is indeed extremely useful, GeoPandas
also allows you to use the spatial index to find
the nearest geometry. The API is similar as above:

In [ ]:
nearest_indices = nyc.sindex.nearest(points)
nearest_indices

You can see that the nearest query returns the indices representation.
If you are interested in how “near” the geometries actually are, the method
can also return distances. In this case, the return format is a tuple of
arrays.

In [ ]:
nearest_indices, distance = nyc.sindex.nearest(points, return_distance=True)
distance